# 3 Beyond A&B Testing

In Question 2 we have explored the straightforward solution to StudyPortals’ inquiry, however, there is
more to explore. In Question 3, we will design an EMM instance that extracts more information out of the StudyPortals situation.


### Multivariate testing

Let's look at what would have happened if StudyPortals had designed four different variations of their webpage. For example, they might not only be interested in the two different button designs that were mentioned, but they might also be interested in knowing the effect of a new logo, a different font, or another colour, at the same time as they are testing their different buttons. As such, we would not be performing a simple A/B test, but instead we would be doing an A/B/C/D test, also known as a multivariate test.

With regular A/B testing, we had targets $t_1$, the binary column representing whether the page visitor merely viewed or also clicked, and $t_2$, the binary column representing whether the visitor was presented version A or B of the buttons. Therefore, the natural choice of EMM instance was the association model class, which allows us to determine the association between two nominal targets.

Now, with A/B/C/D testing, it still makes sense to use the association model, because we are still using two nominal targets. In this case, our targets will be $x$, the binary column representing whether the page visitor merely viewed or also clicked, and $y$, the numerical column representing whether the visitor was presented version A, B, C, or D of the webpage.

Even though we will be using integers (1, 2, 3, 4 for version A, B, C, D respectively) to code the distinct values of $y$, their values will be treated as unordered (nominal).

##### Quality measure

Having fixed the model class, we need to define an appropriate quality measure. To ensure the discovery of subgroups that represent substantial effects within the datasets, a common approach is to craft a quality measure by multiplying two components: one reflecting target deviation, and one reflecting subgroup size.

![Title](Target cross table.png)

##### Target deviation component

For the quality measure component representing the target deviation, we use the target contingency table, shown in Table 1. Now, since this is no longer a 2x2 matrix, we cannot use Yule's Quality Measure here. Instead, we will have to define another appropriate quality measure.

We start by computing the fraction of people who clicked, for each version. This is done as follows: <br>
$a = C_A / (V_A + C_A)$ <br>
$b = C_B / (V_B + C_B)$ <br>
$c = C_C / (V_C + C_C)$ <br>
$d = C_D / (V_D + C_D)$ <br>
This gives us four fractions $a, b, c, d$ (for versions A, B, C and D, respectively), where each fraction is $0$ if no one clicked on the buttons, and $1$ if everyone clicked on the buttons.

Now that we have these four fractions, we can compute their average as follows: <br>
$avg = (a + b + c + d) / 4$ <br>
This gives us the average fraction of people who clicked on the buttons. Still $0$ if no one clicked on any button, and $1$ if everyone clicked on all buttons.

Now, to see how much every version deviated from this average, we take the difference between the fraction for each version and the average. To get the total deviation, we simply add these results together, as follows: <br>
$dev = |a - avg| + |b - avg| + |c - avg| + |d - avg|$ <br>
This gives us a value representing the total deviation from the average. Now, this value is $0$ if every fraction was the same as the average, and $2$ if every fraction was as far from the average as possible ($a = 0$, $b = 0$, $c = 1$, $d = 1$, for example). To get a value between $0$ and $1$ again, we simply divide by two: <br>
$dev = (|a - avg| + |b - avg| + |c - avg| + |d - avg|) / 2$ <br>
Finally, this gives us our quality measure component representing the target devation. A value between $0$ and $1$, which is $0$ if the click rates are the same for each version, and $1$ if the click rates are as different as possible.

##### Subgroup size component

To calculate a quality measure representing the subgroup size, we first calculate the size of each subgroup, and use them to calculate the size of the total population. This is done as follows: <br>
$n_A = V_A + C_A$ <br>
$n_B = V_B + C_B$ <br>
$n_C = V_C + C_C$ <br>
$n_D = V_D + C_D$ <br>
$N = n_A + n_B + n_C + n_D$ <br>

Now that we have these sizes, we can use them to fill in the entropy function as described in [1]. But first, we have to adapt this function slightly to accomodate for our four subgroups, instead of the usual two. We do this as follows:
The regular entropy function: <br>
$H_p = -n/N * log(n/N) - n^c /N * log(n^c /N)$ <br>
Where $n$ is the size of one subgroup, and $n^c$ is its complement, i.e. the size of the other subgroup.

To adapt this entropy function for our use case, we simply change the logarithms into base four logarithms, and add a term for each of our subgroups: <br>
$H_p = -n_A / N * log_4(n_A / N) - n_B / N * log_4(n_B / N) - n_C / N * log_4(n_C / N) - n_D / N * log_4(n_D / N)$ <br>
This function approaches $0$ when one subgroup contains nearly all people, and it results in $1$ when all subgroups are of the same size. Thus rewarding even 25/25/25/25 splits, while punishing subgroups that are tiny or cover the majority of the dataset.

##### Combining the components: Remco's quality measure

Combining the target deviation component and the subgroup size components into an association model class quality measure is as straightforward as multiplying them: <br>
$\varphi _R = dev * H_p$ <br>

With this proposed EMM instance, we can compare the results of more than two variations in our webpage simultaneously. This could be very useful if StudyPortal wanted to know the effectiveness of two different variables at the same time. For instance, they could be experimenting with two different fonts, as well as the two different types buttons mentioned in the exercise.

The model class and quality measure that we have designed are appropriate because they reward subgroups of similar sizes with large deviations in the targets, while punishing subgroups of disproportionate sizes with small deviations in the targets. This ensures the discovery of subgroups that represent substantial effects within the dataset.

### References
[1] Leman, D., Feelders, A., Knobbe, A.J.: Exceptional Model Mining. Proc.
ECML/PKDD (2), 1–16 (2008)